In [ ]:
%pip install -U langchain jq openai chromadb

In [ ]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["monster_source"] = record.get("monster_source")
    metadata["monster_name"] = record.get("monster_name")
    metadata["field"] = record.get("field")

    return metadata

In [ ]:
from langchain.document_loaders import JSONLoader
from pprint import pprint

jq_schema = f'.[]'
loader = JSONLoader(
    file_path='./monster_text.json', jq_schema=jq_schema, content_key="text", metadata_func=metadata_func)
documents = loader.load()

pprint(documents)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)
all_splits = text_splitter.split_documents(documents)
pprint(all_splits)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())